In [1]:
import os
import cv2
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from efficientnet_pytorch import EfficientNet

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout,
    ShiftScaleRotate, CenterCrop, Resize
)
from albumentations.pytorch import ToTensorV2

In [3]:
# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [4]:
class SatelliteDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

        # transforms
        self.transform = transform
        self.default_transform = Compose([
            Resize(224, 224),
            Normalize(),
            ToTensorV2()
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 1]
        # print(img_path)

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            else:
                image = self.default_transform(image=image)['image']
            return image

        mask_rle = self.data.iloc[idx, 2]
        mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
        else:
            augmented = self.default_transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

In [5]:
# U-Net의 기본 구성 요소인 Double Convolution Block을 정의합니다.
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True)
    )
class UNet(nn.Module):
    def __init__(self, backbone_name='efficientnet-b5', classes=1, encoder_weights='imagenet'):
        super(UNet, self).__init__()

        self.backbone = EfficientNet.from_pretrained(backbone_name, weights_path=None)

        self.dconv_down1 = double_conv(3, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.dconv_down4 = double_conv(256, 512)

        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)

        self.conv_last = nn.Conv2d(64, classes, 1)

    def forward(self, x):
        # Backbone feature extraction
        backbone_features = self.backbone.extract_features(x)

        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)

        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)

        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)

        x = self.dconv_down4(x)

        x = self.upsample(x)
        x = torch.cat([x, conv3], dim=1)

        x = self.dconv_up3(x)
        x = self.upsample(x)
        x = torch.cat([x, conv2], dim=1)

        x = self.dconv_up2(x)
        x = self.upsample(x)
        x = torch.cat([x, conv1], dim=1)

        x = self.dconv_up1(x)

        out = self.conv_last(x)

        return out


In [6]:
aug_transform = Compose([
        Resize(224, 224),
        A.HorizontalFlip(p=1.0), # 50% 확률로 이미지를 수평으로 뒤집음
        Normalize(), # 이미지를 정규화
        ToTensorV2() # PyTorch tensor로 변환
    ]
)

dataset = SatelliteDataset(csv_file='./train.csv', transform=aug_transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=6)



model = UNet().to(device)

model = UNet(backbone_name='efficientnet-b5', encoder_weights='imagenet').to(device)

# # loss function과 optimizer 정의
criterion = torch.nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

# training loop
for epoch in range(10):
    model.train()
    epoch_loss = 0
    for images, masks in tqdm(dataloader):
        images = images.float().to(device)
        masks = masks.float().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks.unsqueeze(1))
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {epoch_loss/len(dataloader)}')


aug_transform = Compose([
        Resize(224, 224),
        A.VerticalFlip(p=1.0), # 50% 확률로 이미지를 수직으로 뒤집음
        Normalize(), # 이미지를 정규화
        ToTensorV2() # PyTorch tensor로 변환
    ]
)

dataset = SatelliteDataset(csv_file='./train.csv', transform=aug_transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=6)
# training loop
for epoch in range(10):
    model.train()
    epoch_loss = 0
    for images, masks in tqdm(dataloader):
        images = images.float().to(device)
        masks = masks.float().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks.unsqueeze(1))
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {epoch_loss/len(dataloader)}')

transform = A.Compose(
    [   
        A.Resize(224, 224),
        A.Normalize(),
        ToTensorV2()
    ]
)

dataset = SatelliteDataset(csv_file='./train.csv', transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=6)
# training loop
for epoch in range(10):
    model.train()
    epoch_loss = 0
    for images, masks in tqdm(dataloader):
        images = images.float().to(device)
        masks = masks.float().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks.unsqueeze(1))
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {epoch_loss/len(dataloader)}')

Loaded pretrained weights for efficientnet-b5
Loaded pretrained weights for efficientnet-b5


100%|██████████| 447/447 [02:00<00:00,  3.72it/s]


Epoch 1, Loss: 0.18327906186081952


100%|██████████| 447/447 [01:59<00:00,  3.75it/s]


Epoch 2, Loss: 0.13475662249846745


...

Epoch 8, Loss: 0.05356255468289457


100%|██████████| 447/447 [01:59<00:00,  3.74it/s]


Epoch 9, Loss: 0.05591017603707527


100%|██████████| 447/447 [01:59<00:00,  3.74it/s]

Epoch 10, Loss: 0.05335654785869105


## **픽셀의 숫자가 특정 개수 이하면 -1**

In [7]:
model_name = "UNet_wt.pth"
torch.save(model.state_dict(), "./" + model_name)

**컨투어 검출후 에러 삭제**

In [8]:
# # 컨투어 검출하기
# def contoursDelet(mask):
#   mask = np.array(mask)
#   contours, hierarchy = cv2.findContours(mask.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

#   # cv2.drawContours(img_copy, contours, -1, (0, 0, 255), 1)

#   # Threshold를 지정하세요.
#   threshold = 50

#   # Create an empty mask to store the pixels to be removed
#   remove_mask = np.zeros_like(mask)

#   # 각 컨투어를 순회하며 면적 계산
#   for idx, contour in enumerate(contours):
#       # 현재 컨투어가 최상위 컨투어이고(부모가 없고), 하위 컨투어(자식이)가 없으면 단일 구조
#       if hierarchy[0][idx][3] == -1 and hierarchy[0][idx][2] == -1:
#           area = cv2.contourArea(contour)
#           print(area)
#           if area < threshold:
#               cv2.drawContours(remove_mask, [contour], -1, 255, thickness=cv2.FILLED)

#       # 현재 컨투어가 최상위 컨투어이고, 하위 컨투어(자식이)가 있으면 뚫린 구조
#       elif hierarchy[0][idx][3] == -1 and hierarchy[0][idx][2] != -1:
#           outer_area = cv2.contourArea(contour)
#           inner_area = cv2.contourArea(contours[hierarchy[0][idx][2]])
#           print(outer_area - inner_area)
#           if (outer_area - inner_area) < threshold:
#               cv2.drawContours(remove_mask, [contour], -1, 255, thickness=cv2.FILLED)

#   # Remove the small areas from the original mask
#   final_mask = cv2.bitwise_and(mask, cv2.bitwise_not(remove_mask))

#   return final_mask


In [9]:
transform = A.Compose(
    [
        A.Resize(112, 112),
        A.Normalize(),
        ToTensorV2()
    ]
)

In [10]:
test_dataset = SatelliteDataset(csv_file='./test.csv', transform=transform, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=6)

In [11]:
model.load_state_dict(torch.load('./UNet_wt.pth', map_location=torch.device('cpu')))
model = model.to(device)

In [12]:
with torch.no_grad():
    model.eval()
    result = []
    for images in tqdm(test_dataloader):
        images = images.float().to(device)
        
        outputs = model(images)
        masks = torch.sigmoid(outputs).cpu().numpy()
        masks = np.squeeze(masks, axis=1)
        masks = (masks > 0.3).astype(np.uint8) # Threshold = 0.35
        
        for i in range(len(images)):
            mask_resized = cv2.resize(masks[i], (224, 224))
            mask_rle = rle_encode(mask_resized)
            if mask_rle == '': # 예측된 건물 픽셀이 아예 없는 경우 -1
                result.append(-1)
            else:
                result.append(mask_rle)

100%|██████████| 3790/3790 [02:57<00:00, 21.39it/s]


In [13]:
submit = pd.read_csv('./sample_submission.csv')
submit['mask_rle'] = result

In [14]:
submit.to_csv('./submit.csv', index=False)